## Least Recently Used Cache
We have briefly discussed caching as part of a practice problem while studying hash maps.

The lookup operation (i.e., `get(`)) and `put()` / `set()` is supposed to be fast for a cache memory.

While doing the `get()` operation, if the entry is found in the cache, it is known as a cache hit. If, however, the entry is not found, it is known as a cache miss.

When designing a cache, we also place an upper bound on the size of the cache. If the cache is full and we want to add a new entry to the cache, we use some criteria to remove an element. After removing an element, we use the `put()` operation to insert the new element. The remove operation should also be fast.

For our first problem, the goal will be to design a data structure known as a Least Recently Used (LRU) cache. An LRU cache is a type of cache in which we remove the least recently used entry when the cache memory reaches its limit. For the current problem, consider both get and set operations as an use operation.

Your job is to use an appropriate data structure(s) to implement the cache.

In case of a cache hit, your `get()` operation should return the appropriate value.
In case of a cache miss, your `get()` should return -1.
While putting an element in the cache, your `put()` / `set()` operation must insert the element. If the cache is full, you must write code that removes the least recently used entry first and then insert the element.
All operations must take O(1) time.

For the current problem, you can consider the size of cache = 5.

Here is some boiler plate code and some example test cases to get you started on this problem:

## Writeup
I conceived of an LRU cache as a combination of a Queue and a Map.  
A map allows a fast (constant time) lookup operation, and a queue allows us to keep an order without an index.  
A queue built from a doubly linked list can perform insertions and deletions in constant time.  
I decided to use the built-in Python `dict` type as a map, and then build a queue as a separate class. I then wrote an LRU class that has its own map and its own (DLL) queue working in concert.  
Incorporating the traits of queue directly into the LRU cache class would have been shorter, but I find it more explicit and easier to conceptualize when those functions exist separately.  
I assumed that actually writing a hash function was outside the scope of this problem.  
I'm fairly confident that the `set()` method operates in constant time, as there are no loops in the code. We are just looking up entries in a dict and overwriting the values if necessary. Any time we "use" an element in the cache, either by or putting it in the cash or getting it out, we need to reorder the elements such that the most recently used element moves to the tail. This necessitates deleting a node and enqueueing it. Because a linked-list data structure does not have an index, there is no need to reindex it when adding or removing elements. This means there is no need to iterate over the entire queue, and the runtime is independent of the size of the queue.

In [1]:
class Node(object):
    
    def __init__(self, value, key=None):
        self.key = key
        self.value = value
        self.next = None
        self.previous = None

class Queue():
    
    def __init__(self):
        self.size = 0
        self.head = None
        self.tail = None
    
    def enque(self, new_node): #add new item to tail
        if self.head is None:
            self.head = new_node
            self.tail = new_node
        else:
            new_node.previous = self.tail #backlink current tail to new node
            self.tail.next = new_node #frontlink new node to current tail
            self.tail = new_node #shift tail
            self.tail.next = None #in case new node already has a `next` node.
        self.size += 1
    
    def deque(self): #pop off oldest item from head
        if self.size > 0:
            old_head = self.head
            self.head = old_head.next
            self.size -= 1
            return old_head
        else:
            return None
    
    def _delete(self, node): #remove node arbitrarily; needs to handle three cases for a doubly-linked list
        if self.size == 1: #if only one node, create an empty queue.
            self.head = None
            self.tail = None
        elif node == self.head:
            self.head = self.head.next
            self.head.previous = None
        elif node == self.tail:
            self.tail = self.tail.previous
            self.tail.next = None
        else:
            left_node = node.previous
            right_node = node.next
            left_node.next = right_node
            right_node.previous = left_node
        self.size -= 1
        return

class LRU_Cache(object):
    
    def __init__(self, capacity):
        # Initialize class variables
        self.capacity = capacity
        self.map = {}
        self.q = Queue()
    
    def put(self, hashcode, value):
        if hashcode in self.map.keys():
            old_node = self.map[hashcode]
            old_node.value = value #update mapping
            self.q._delete(old_node) #remove node from current place in queue
            self.q.enque(old_node) #add node onto tail of queue, now last-in.
            
        else:
            if self.q.size == self.capacity:
                node_to_remove = self.q.deque()
                hashcode_to_remove = node_to_remove.key #this is why Node class has a `key` attribute.
                del self.map[hashcode_to_remove] #remove first-in item from mapping
            new_node = Node(value=value, key=hashcode) # create new node
            self.map[hashcode] = new_node #create mapping
            self.q.enque(new_node) #add new node to end of queue.
        
    def get(self, hashcode):
        if hashcode in self.map.keys():
            node = self.map[hashcode] # retrieve node from hashmap
            self.q._delete(node) # remove node from current position in queue
            self.q.enque(node) #add node onto tail of queue
            return node.value
        else:
            return -1

In [4]:
def verbose_check(our_cache):
    print("cache map: ", {hashmap: node.value for hashmap, node in our_cache.map.items()}, "\n")
    node = our_cache.q.head
    counter = 1
    print("queue: ")
    while node:
        print(f"pos: {counter}", ";", node.key, ":", node.value)
        node = node.next
        counter += 1

In [5]:
our_cache = LRU_Cache(5)

our_cache.put(1, 5);
our_cache.put(2, 10);
our_cache.put(3, 15);
our_cache.put(4, 20);

verbose_check(our_cache)

cache map:  {1: 5, 2: 10, 3: 15, 4: 20} 

queue: 
pos: 1 ; 1 : 5
pos: 2 ; 2 : 10
pos: 3 ; 3 : 15
pos: 4 ; 4 : 20


In [6]:
assert(our_cache.get(1) == 5) #retrieve first-in item

assert(our_cache.q.tail.value == 5) #should now be last-in item

assert(our_cache.q.head.value == 10) #second item should now be first

verbose_check(our_cache)

assert(our_cache.get(7) == -1) # not in the cache

assert({hashmap: node.value for hashmap, node in our_cache.map.items()} == {1: 5, 2: 10, 3: 15, 4: 20}) #assert all key-value combinations

cache map:  {1: 5, 2: 10, 3: 15, 4: 20} 

queue: 
pos: 1 ; 2 : 10
pos: 2 ; 3 : 15
pos: 3 ; 4 : 20
pos: 4 ; 1 : 5


### Check capacity

In [7]:
our_cache.put(5, 25) # brings us up to capacity of cache

our_cache.put(6, 30) # brings us past capacity of cache, pushing first item off head.

verbose_check(our_cache)

assert(our_cache.q.tail.value == 30)

assert(our_cache.q.head.value == 15)

cache map:  {1: 5, 3: 15, 4: 20, 5: 25, 6: 30} 

queue: 
pos: 1 ; 3 : 15
pos: 2 ; 4 : 20
pos: 3 ; 1 : 5
pos: 4 ; 5 : 25
pos: 5 ; 6 : 30


In [19]:
our_cache.put(5, "cat") #overwrite current value
assert(our_cache.q.head.value == 15) #head should not have changed
assert((our_cache.q.tail.key == 5) & (our_cache.q.tail.value == "cat")) #updated value should now be at tail.

In [20]:
verbose_check(our_cache)

cache map:  {1: 5, 3: 15, 4: 20, 5: 'cat', 6: 30} 

queue: 
pos: 1 ; 3 : 15
pos: 2 ; 4 : 20
pos: 3 ; 1 : 5
pos: 4 ; 6 : 30
pos: 5 ; 5 : cat


In [22]:
# Solution from 
# https://leetcode.com/problems/lru-cache/discuss/309546/Python%3A-Easy-to-follow-dict-%2B-doubly-linked-list-solution

class ListNode:
    def __init__(self, key, val):
        self.prev = None
        self.next = None
        self.key = key
        self.val = val

class LRUCache(object):

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.capacity = capacity
        self.kv_store = {}
        self.head = None
        self.last = None
        self.cur_len = 0

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        val = self._get(key)
        if val == -1:
            return val
        
        self._remove(key)
        self._push(key, val)
        n = self.head
        return val

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: None
        """
        
        # if it exists already, take it out then push it to the front
        if self._get(key) != -1: 
            self._remove(key)
            self._push(key, value)
            return
        
        if self.cur_len == self.capacity:
            self._remove(self.last.key)
        
        self._push(key, value)
        
    def _get(self, key):
        node = self.kv_store.get(key)
        if node:
            return node.val
        else:
            return -1
    
    # push to beginning of queue
    def _push(self, key, val):
        new_node = ListNode(key, val)
        if self.head:
            new_node.next = self.head
            self.head.prev = new_node
            self.head = new_node
        else:
            self.head = new_node
            self.last = new_node
        
        self.kv_store[key] = new_node
        self.cur_len += 1
    
    def _remove(self, key):
        rem_node  = self.kv_store[key]
        del self.kv_store[key]
        prev_node = rem_node.prev
        next_node = rem_node.next
        if prev_node:
            prev_node.next = next_node
        if next_node:
            next_node.prev = prev_node
        
        if rem_node == self.head:
            self.head = next_node
        
        if rem_node == self.last:
            self.last = prev_node
        
        self.cur_len -= 1

In [ ]:
# Solution from :
# https://leetcode.com/problems/lru-cache/discuss/46121/lru-cache-implementation-in-python-w-explanation
from __future__ import print_function

"""
QNode -> holds key and value; as well as pointers to previous and next nodes.
"""
class QNode(object):
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None


    def __str__(self):
        return "(%s, %s)" % (self.key, self.value)


class LRUCache(object):
    def __init__(self, capacity):
        """
        :type capacity: int
        """
        if capacity <= 0:
            raise ValueError("capacity > 0")
        self.hash_map = {}

        # No explicit doubly linked queue here (you may create one yourself)
        self.head = None
        self.end = None

        self.capacity = capacity
        self.current_size = 0


    # PUBLIC


    def get(self, key):
        """
        :rtype: int
        """
        if key not in self.hash_map:
            return -1
        
        node = self.hash_map[key]

        # small optimization (1): just return the value if we are already looking at head
        if self.head == node:
            return node.value
        self.remove(node)
        self.set_head(node)
        return node.value
        

    def set(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: nothing
        """
        if key in self.hash_map:
            node = self.hash_map[key]
            node.value = value

            # small optimization (2): update pointers only if this is not head; otherwise return
            if self.head != node:
                self.remove(node)
                self.set_head(node)
        else:
            new_node = QNode(key, value)
            if self.current_size == self.capacity:
                del self.hash_map[self.end.key]
                self.remove(self.end)
            self.set_head(new_node)
            self.hash_map[key] = new_node


    # PRIVATE

    def set_head(self, node):
        if not self.head:
            self.head = node
            self.end = node
        else:
            node.prev = self.head
            self.head.next = node
            self.head = node
        self.current_size += 1


    def remove(self, node):
        if not self.head:
            return

        # removing the node from somewhere in the middle; update pointers
        if node.prev:
            node.prev.next = node.next
        if node.next:
            node.next.prev = node.prev

        # head = end = node
        if not node.next and not node.prev:
            self.head = None
            self.end = None

        # if the node we are removing is the one at the end, update the new end
        # also not completely necessary but set the new end's previous to be NULL
        if self.end == node:
            self.end = node.next
            self.end.prev = None
        self.current_size -= 1
        return node


    def print_elements(self):
        n = self.head
        print("[head = %s, end = %s]" % (self.head, self.end), end=" ")
        while n:
            print("%s -> " % (n), end = "")
            n = n.prev
        print("NULL")